# Valuing Caps and Floors

We value caps and floors using Black's model

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 16:00 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Building a Ibor Curve

In [3]:
valuation_date = Date(6, 6, 2018)

In [4]:
spot_days = 0
settlement_date = valuation_date
depoDCCType = DayCountTypes.THIRTY_E_360_ISDA

depos = []
depo = IborDeposit(settlement_date, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settlement_date, "2M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settlement_date, "3M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settlement_date, "6M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settlement_date, "9M", 0.0230, depoDCCType); depos.append(depo)

In [5]:
accrual = DayCountTypes.THIRTY_E_360
freq = FrequencyTypes.SEMI_ANNUAL
longEnd = DateGenRuleTypes.BACKWARD
swapType = SwapTypes.PAY

spot_days = 2
settlement_date = valuation_date.add_weekdays(spot_days)

swaps = []
swap = IborSwap(settlement_date, "1Y", swapType, 0.0250, freq, accrual); swaps.append(swap)
swap = IborSwap(settlement_date, "2Y", swapType, 0.0255, freq, accrual); swaps.append(swap)
swap = IborSwap(settlement_date, "3Y", swapType, 0.0260, freq, accrual); swaps.append(swap)
swap = IborSwap(settlement_date, "4Y", swapType, 0.0265, freq, accrual); swaps.append(swap)
swap = IborSwap(settlement_date, "5Y", swapType, 0.0270, freq, accrual); swaps.append(swap)

In [6]:
libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

## Creating a Cap and Floor

In [7]:
capType = FinCapFloorTypes.CAP
floorType = FinCapFloorTypes.FLOOR

In [8]:
strike_rate = 0.02

In [9]:
cap = IborCapFloor(settlement_date, "2Y", capType, strike_rate)
flr = IborCapFloor(settlement_date, "2Y", floorType, strike_rate)

In [10]:
print(cap)

OBJECT TYPE: IborCapFloor
START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
STRIKE COUPON: 2.0
OPTION TYPE: FinCapFloorTypes.CAP
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA


In [11]:
print(flr)

OBJECT TYPE: IborCapFloor
START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
STRIKE COUPON: 2.0
OPTION TYPE: FinCapFloorTypes.FLOOR
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA


# Valuation

### Black's Model

We start with Black's model with 25% volatility

In [12]:
model = Black(0.25)

In [13]:
cap.value(valuation_date, libor_curve, model)

11364.358596519098

In [14]:
flr.value(valuation_date, libor_curve, model)

830.1408120621414

In [15]:
cap.print_leg()

START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
OPTION TYPE FinCapFloorTypes.CAP
STRIKE (%): 2.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE 06-JUN-2018
PAYMENT_DATE     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
    08-JUN-2018          -         -            -     1.000000            -            -
    10-SEP-2018  0.2555556   2.29838       757.99     0.994034       757.99       757.99
    10-DEC-2018  0.2500000   2.28739       710.13     0.988382       760.00      1517.99
    08-MAR-2019  0.2444444   2.29062       698.24     0.982878       815.49      2333.48
    10-JUN-2019  0.2555556   3.08483      2703.56     0.975190      2714.74      5048.22
    09-SEP-2019  0.2472222   2.60810      1456.67     0.968943      1558.15      6606.37
    09-DEC-2019  0.2500000   2.57912      1393.85     0.962735      1544.42      8150.79
    09-MAR-2020  0.2500000   2.57912      1384.92     0.956568      1573.61      

# Alternative Models

## Shifted Black

Shifted Black gives the same pdf at F+S as Black does at F. So if we want to have the PDF for F=0.25 at -0.25 because rates are negative then you need to set F=-0.50.

In [17]:
model = BlackShifted(0.25, -0.005)

In [18]:
cap.value(valuation_date, libor_curve, model)

10931.197435082659

In [19]:
flr.value(valuation_date, libor_curve, model)

396.9796506257012

The floor has increased in value as the downside risk is greater.

## SABR Model

In [20]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573

In [21]:
model = SABR(alpha, beta, rho, nu)

In [22]:
cap.value(valuation_date, libor_curve, model)

11467.46038874608

In [23]:
flr.value(valuation_date, libor_curve, model)

933.2426042891235

## Shifted SABR Model

In [24]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573; shift = -0.005

In [25]:
model = SABRShifted(alpha, beta, rho, nu, shift)

In [26]:
cap.value(valuation_date, libor_curve, model)

11834.498534977098

In [27]:
flr.value(valuation_date, libor_curve, model)

1300.2807505201408

## Hull White Model

In [28]:
sigma = 0.01; alpha = 0.005

In [29]:
model = HWTree(sigma, alpha)

In [30]:
cap.value(valuation_date, libor_curve, model)

14292.202257996401

In [31]:
flr.value(valuation_date, libor_curve, model)

2773.738790003152

Copyright (c) 2020 Dominic O'Kane